In [1]:
import torch
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.datasets import fetch_20newsgroups

In [2]:
x = torch.IntTensor([1,3,6]) 
y = torch.IntTensor([1,1,1])
result = x + y
print(result)

tensor([2, 4, 7], dtype=torch.int32)


In [3]:
categories = ["comp.graphics","sci.space","rec.sport.baseball"]
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)

print('total texts in train:',len(newsgroups_train.data))
print('total texts in test:',len(newsgroups_test.data))

total texts in train: 1774
total texts in test: 1180


In [4]:
vocab = Counter()

for text in newsgroups_train.data:
    for word in text.split(' '):
        vocab[word.lower()]+=1
        
for text in newsgroups_test.data:
    for word in text.split(' '):
        vocab[word.lower()]+=1
        
total_words = len(vocab)

def get_word_2_index(vocab):
    word2index = {}
    for i,word in enumerate(vocab):
        word2index[word.lower()] = i
        
    return word2index

word2index = get_word_2_index(vocab)

In [5]:
def get_batch(df,i,batch_size):
    batches = []
    results = []
    texts = df.data[i*batch_size:i*batch_size+batch_size]
    categories = df.target[i*batch_size:i*batch_size+batch_size]
    for text in texts:
        layer = np.zeros(total_words,dtype=float)
        for word in text.split(' '):
            layer[word2index[word.lower()]] += 1
        
        batches.append(layer)
        
    for category in categories:
        index_y = -1
        if category == 0:
            index_y = 0
        elif category == 1:
            index_y = 1
        else:
            index_y = 2
        results.append(index_y)
            
     
    return np.array(batches),np.array(results)

In [6]:
# Parameters
learning_rate = 0.01
num_epochs = 10
batch_size = 150
display_step = 1

# Network Parameters
hidden_size = 100      # 1st layer and 2nd layer number of features
input_size = total_words # Words in vocab
num_classes = 3         # Categories: graphics, sci.space and baseball

In [7]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [8]:
class OurNet(nn.Module):
     def __init__(self, input_size, hidden_size, num_classes):
        super(OurNet, self).__init__()
        self.layer_1 = nn.Linear(input_size,hidden_size, bias=True)
        self.relu = nn.ReLU()
        self.layer_2 = nn.Linear(hidden_size, hidden_size, bias=True)
        self.output_layer = nn.Linear(hidden_size, num_classes, bias=True)
 
     def forward(self, x):
        out = self.layer_1(x)
        out = self.relu(out)
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.output_layer(out)
        return out

In [9]:
# input [batch_size, n_labels]
# output [max index for each item in batch, ... ,batch_size-1]
loss = nn.CrossEntropyLoss()
input = Variable(torch.randn(2, 5), requires_grad=True)
print(">>> batch of size 2 and 5 possible classes")
print(input)
target = Variable(torch.LongTensor(2).random_(5))
print(">>> array of size 'batch_size' with the index of the maxium label for each item")
print(target)
output = loss(input, target)
output.backward()

>>> batch of size 2 and 5 possible classes
tensor([[ 0.2811, -1.2138, -0.7723,  1.4370,  1.0231],
        [-0.4353, -1.2512,  0.5136,  1.2858,  0.3641]], requires_grad=True)
>>> array of size 'batch_size' with the index of the maxium label for each item
tensor([4, 0])


In [10]:
net = OurNet(input_size, hidden_size, num_classes)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)  

# Train the Model
for epoch in range(num_epochs):
    total_batch = int(len(newsgroups_train.data)/batch_size)
    # Loop over all batches
    for i in range(total_batch):
        batch_x,batch_y = get_batch(newsgroups_train,i,batch_size)
        articles = Variable(torch.FloatTensor(batch_x))
        labels = Variable(torch.LongTensor(batch_y))
        
#         print("articles",articles)
#         print(batch_x, labels)
#         print("size labels",labels.size())
                
        
        # Forward + Backward + Optimize        
        outputs = net(articles)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()  # zero the gradient buffer
        loss.backward()
        optimizer.step()
        
        if (i+1) % 4 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(newsgroups_train.data)//batch_size, loss))

Epoch [1/10], Step [4/11], Loss: 0.9184
Epoch [1/10], Step [8/11], Loss: 0.3291
Epoch [2/10], Step [4/11], Loss: 0.2339
Epoch [2/10], Step [8/11], Loss: 0.0093
Epoch [3/10], Step [4/11], Loss: 0.0657
Epoch [3/10], Step [8/11], Loss: 0.3072
Epoch [4/10], Step [4/11], Loss: 0.0037
Epoch [4/10], Step [8/11], Loss: 0.0008
Epoch [5/10], Step [4/11], Loss: 0.0001
Epoch [5/10], Step [8/11], Loss: 0.0000
Epoch [6/10], Step [4/11], Loss: 0.0003
Epoch [6/10], Step [8/11], Loss: 0.0000
Epoch [7/10], Step [4/11], Loss: 0.0000
Epoch [7/10], Step [8/11], Loss: 0.0000
Epoch [8/10], Step [4/11], Loss: 0.0000
Epoch [8/10], Step [8/11], Loss: 0.0000
Epoch [9/10], Step [4/11], Loss: 0.0000
Epoch [9/10], Step [8/11], Loss: 0.0000
Epoch [10/10], Step [4/11], Loss: 0.0000
Epoch [10/10], Step [8/11], Loss: 0.0000


In [ ]:
# Test the Model
correct = 0
total = 0
total_test_data = len(newsgroups_test.target)
print(total_test_data)

batch_x_test,batch_y_test = get_batch(newsgroups_test,0,total_test_data)
articles = Variable(torch.FloatTensor(batch_x_test))
labels = torch.LongTensor(batch_y_test)

outputs = net(articles)
print(outputs)
_, predicted = torch.max(outputs.data, 1)
total += labels.size(0)
correct += (predicted == labels).sum()
    
print('Accuracy of the network on the 1180 test articles: %d %%' % (100 * correct / total))

1180


In [ ]:
newsgroups_test.target

In [ ]:
np.argmax(outputs.data, axis=1)

In [ ]:
_, predicted = torch.max(outputs.data, 1)

In [ ]:
_

In [ ]:
predicted